In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from service.utils.spark import get_spark_session
# spark = SparkSession.builder.getOrCreate()
spark = get_spark_session(dev=True)
test_namespace = 'warehousedev.gold.test'
spark.conf.get('spark.sql.catalog.warehousedev.s3.region')
spark.sql(f"CREATE NAMESPACE IF NOT EXISTS {test_namespace}")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/09/19 11:28:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


DataFrame[]

In [2]:
def write_iceberg(df, full_table_identifier):
    writer = \
        df.writeTo(full_table_identifier)

    if not spark.catalog.tableExists(full_table_identifier):
        writer.create()
    else:
        writer.overwritePartitions()

In [3]:
# silver
order_status_df = spark.read.csv("s3a://warehousedev/bronze/tsv/order_status.tsv", header=True, sep='\t')

delivered_customer_order_id = order_status_df.filter(order_status_df.status == 'delivered_customer').select(['order_id'])
delivered_customer_order_df = order_status_df.join(delivered_customer_order_id, on='order_id', how='inner')
status_list = ['purchase', 'delivered_customer']
delivered_customer_order_df = delivered_customer_order_df.filter(F.col('status').isin(status_list))

pivoted_df = delivered_customer_order_df.groupBy('order_id') \
    .pivot('status', ['purchase', 'delivered_customer']) \
    .agg(F.first('timestamp'))

delivered_order_df = pivoted_df.withColumnsRenamed({
    'purchase': 'purchase_date',
    'delivered_customer': 'delivery_date'
})
    
ed_df = spark.read.csv("s3a://warehousedev/bronze/tsv/estimated_delivery_date.tsv", header=True, sep='\t')
delivered_order_df = delivered_order_df.join(ed_df, on='order_id', how='inner')
payment_df = spark.read.csv("s3a://warehousedev/bronze/tsv/payment.tsv", header=True, sep='\t')
payment_info_df = payment_df.select(['order_id', 'customer_id']).dropDuplicates()

# payment와 order_status에 모두 있는 order_id만 남김
clean_order_df = delivered_order_df.join(payment_info_df, on='order_id', how='inner')

format_string1 = 'yyyy-MM-dd HH:mm:ss.SSSSSS'
format_string2 = 'yyyy-MM-dd HH:mm:ss'
clean_order_df = clean_order_df.withColumns({
    'purchase_date': F.to_timestamp(F.col('purchase_date'), format_string1),
    'delivery_date': F.to_timestamp(F.col('delivery_date'), format_string1),
    'estimated_delivery_date': F.to_timestamp(F.col('estimated_delivery_date'), format_string2)
    })
clean_order_df = clean_order_df.orderBy(F.col('purchase_date').asc())
# clean_order_df.show(truncate=False)
write_iceberg(clean_order_df, f"{test_namespace}.clean_order")

25/09/19 11:28:52 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


In [4]:
# gold
delivery_stats = clean_order_df \
    .withColumn(
        'delivery_lead_time',
        F.datediff(F.col('delivery_date'), F.col('purchase_date'))) \
    .withColumn(
        'is_late',
        F.when(F.col('delivery_date') <= F.col('estimated_delivery_date'), False)
        .otherwise(True)
    )
# delivery_stats.show()
write_iceberg(delivery_stats, f"{test_namespace}.delivery_stats")

In [5]:
# silver
order_item_df = spark.read.csv("s3a://warehousedev/bronze/tsv/order_item.tsv", header=True, sep='\t')
product_df = spark.read.csv("s3a://warehousedev/bronze/tsv/product.tsv", header=True, sep='\t')

taget_category = 'health_beauty'
taget_category_product = product_df.filter(product_df.category == taget_category).select(['product_id', 'category'])
taget_category_order = taget_category_product.join(order_item_df, on='product_id', how='inner')
taget_category_order = taget_category_order.sort('product_id')

# 하나의 order에 여러 item이 있을 수 있으므로, order_item_id는 남겨둔다.
# `total_price` 는 상품가와 배송비를 합산한 가격
taget_category_order = taget_category_order.withColumn('total_price', F.round(F.col('price') + F.col('freight_value'), 4))
taget_category_order = taget_category_order.drop('shipping_limit_date', 'price', 'freight_value')
clean_category_order_df = taget_category_order.join(clean_order_df.select(['order_id']), on='order_id', how='inner')
# clean_category_order_df.show(truncate=False)
write_iceberg(clean_category_order_df, f"{test_namespace}.clean_category_order")


In [6]:
# gold
sale_stats = clean_category_order_df.groupBy('product_id').agg(
    F.count('order_id').alias('order_count'),
    F.round(F.sum(F.col('total_price')), 4).alias('total_sales')
    ).orderBy(F.col('order_count').desc())

sale_stats = sale_stats.withColumn('mean_sale', F.round(F.col('total_sales') / F.col('order_count'), 4))
# sale_stats.show(truncate=False)
write_iceberg(sale_stats, f"{test_namespace}.sale_stats")

In [7]:
# silver: review metadata
review_df = spark.read.csv("s3a://warehousedev/bronze/tsv/review.tsv", header=True, sep='\t')

review_metadat_df = review_df.drop('review_comment_title', 'review_comment_message')
clean_review_metadata_df = review_metadat_df.join(clean_category_order_df.select('order_id', 'product_id'), on=['order_id'], how='inner')
clean_review_metadata_df = clean_review_metadata_df.orderBy('product_id')

format_string = "yyyy-MM-dd HH:mm:ss"
clean_review_metadata_df = clean_review_metadata_df \
    .withColumn('review_creation_date', F.to_timestamp(F.col('review_creation_date'), format_string)) \
    .withColumn('review_answer_timestamp', F.to_timestamp(F.col('review_answer_timestamp'), format_string))

clean_review_metadata_df = clean_review_metadata_df.withColumn('answer_lead_time', F.datediff(F.col('review_answer_timestamp'), F.col('review_creation_date')) )

clean_review_metadata_df = clean_review_metadata_df.drop('review_creation_date', 'review_answer_timestamp')
# clean_review_metadata_df.show()
write_iceberg(clean_review_metadata_df, f"{test_namespace}.clean_review_metadata")

In [8]:
for file_name in ['geolocation', 'customer', 'seller']:
    print(file_name)
    tmp_df = spark.read.csv(f"s3a://warehousedev/bronze/tsv/{file_name}.tsv", header=True, sep='\t')
    write_iceberg(tmp_df, f"{test_namespace}.{file_name}")

geolocation
customer
seller
